

---



In [1]:
#@title <h1>Step1: Setup Wav2Lip</h1>
# @markdown * Install dependency
# @markdown   * Download pretrained model
!rm -rf /content/sample_data
!mkdir /content/sample_data

!git clone https://github.com/zabique/Wav2Lip

#download the pretrained model
!wget 'https://iiitaphyd-my.sharepoint.com/personal/radrabha_m_research_iiit_ac_in/_layouts/15/download.aspx?share=EdjI7bZlgApMqsVoEUUXpLsBxqXbn5z8VTmoxp55YNDcIA' -O '/content/Wav2Lip/checkpoints/wav2lip_gan.pth'
a = !pip install https://raw.githubusercontent.com/AwaleSajil/ghc/master/ghc-1.0-py3-none-any.whl

# !pip uninstall tensorflow tensorflow-gpu
!cd Wav2Lip && pip install -r requirements.txt

#download pretrained model for face detection
!wget "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth" -O "/content/Wav2Lip/face_detection/detection/sfd/s3fd.pth"

!pip install -q youtube-dl
!pip install ffmpeg-python
!pip install librosa==0.9.1

#this code for recording audio
"""
To write this piece of code I took inspiration/code from a lot of places.
It was late night, so I'm not sure how much I created or just copied o.O
Here are some of the possible references:
https://blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/
https://stackoverflow.com/a/18650249
https://hacks.mozilla.org/2014/06/easy-audio-capture-with-the-mediarecorder-api/
https://air.ghost.io/recording-to-an-audio-file-using-html5-and-js/
https://stackoverflow.com/a/49019356
"""
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

from IPython.display import clear_output
clear_output()
print("\nDone")


Done


下面两块不用动



---





---



In [4]:

!cd Wav2Lip && python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face "/content/drive/MyDrive/Moc影片摘要生成/class01.mp4" --audio "/content/drive/MyDrive/Moc影片摘要生成/class01.WAV"

Using cuda for inference.
Reading video frames...
^C


In [ ]:
#@title 3.Play result video -  50% scaling
from IPython.display import HTML
from base64 import b64encode
mp4 = open('/content/Wav2Lip/results/result_voice.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""
<video width="50%" height="50%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")

连接谷歌云端

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


友喜

In [ ]:
import os
import subprocess
import shutil

# 设置视频和音频输入文件夹路径
video_folder = '/content/drive/MyDrive/LEEvideo'
audio_folder = '/content/drive/MyDrive/LEEvoice'
output_folder = '/content/output'
wav2lip_dir = '/content/Wav2Lip'  # Wav2Lip目录的路径
wav2lip_checkpoint = '/content/Wav2Lip/checkpoints/wav2lip_gan.pth'

# 确保输出文件夹存在
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 获取视频和音频文件夹中的所有文件名（不包括文件扩展名）
video_files = {os.path.splitext(file)[0] for file in os.listdir(video_folder) if file.endswith(".mp4")}
audio_files = {os.path.splitext(file)[0] for file in os.listdir(audio_folder) if file.endswith(".wav")}

# 找出匹配的文件名
matched_files = video_files.intersection(audio_files)

# 对于每个匹配的文件名，执行嘴型同步处理
for file_name in matched_files:
    video_path = os.path.join(video_folder, f"{file_name}.mp4")
    audio_path = os.path.join(audio_folder, f"{file_name}.wav")
    temp_output_video_path = os.path.join(wav2lip_dir, 'results/result_voice.mp4')
    final_output_video_path = os.path.join(output_folder, f"{file_name}.mp4")

    # 构建Wav2Lip命令
    command = [
        'python', 'inference.py',
        '--checkpoint_path', wav2lip_checkpoint,
        '--face', video_path,
        '--audio', audio_path,
        '--outfile', temp_output_video_path
    ]

    # 切换到Wav2Lip目录并运行命令
    os.chdir(wav2lip_dir)
    subprocess.run(command)

    # 将结果视频文件移动并重命名到输出文件夹
    if os.path.exists(temp_output_video_path):
        shutil.move(temp_output_video_path, final_output_video_path)

print("处理完成！")

处理完成！


AITT

In [ ]:
import os
import subprocess
import shutil
import re  # 导入正则表达式库

# 设置媒体输入文件夹路径和输出文件夹路径
media_folder = '/content/drive/MyDrive/Moc影片摘要生成/seg1'  # 包含视频和音频文件的文件夹
google_drive_output_folder = '/content/drive/MyDrive/Moc影片摘要生成/class1'  # 输出文件夹在Google云端硬盘中的路径
wav2lip_dir = '/content/Wav2Lip'  # Wav2Lip目录的路径
wav2lip_checkpoint = '/content/Wav2Lip/checkpoints/wav2lip_gan.pth'

# 确保Google云端硬盘输出文件夹存在
if not os.path.exists(google_drive_output_folder):
    os.makedirs(google_drive_output_folder)

# 从文件夹中获取所有视频文件名
video_files = {file for file in os.listdir(media_folder) if file.endswith(".mp4")}
# 从文件夹中获取所有音频文件名
audio_files = {file for file in os.listdir(media_folder) if file.endswith(".wav")}

# 提取文件名中的数字并进行匹配
for video_file in video_files:
    video_number = re.search(r'\d+', video_file).group()  # 提取视频文件名中的数字
    for audio_file in audio_files:
        if re.search(r'\d+', audio_file).group() == video_number:  # 检查数字是否匹配
            video_path = os.path.join(media_folder, video_file)
            audio_path = os.path.join(media_folder, audio_file)
            file_name = f"segment{video_number}"
            temp_output_video_path = os.path.join(wav2lip_dir, 'results/result_voice.mp4')
            final_output_video_path = os.path.join(google_drive_output_folder, f"{file_name}.mp4")

            # 构建Wav2Lip命令
            command = [
                'python', 'inference.py',
                '--checkpoint_path', wav2lip_checkpoint,
                '--face', video_path,
                '--audio', audio_path,
                '--outfile', temp_output_video_path
            ]

            # 切换到Wav2Lip目录并运行命令
            os.chdir(wav2lip_dir)
            subprocess.run(command)

            # 将结果视频文件移动并重命名到Google云端硬盘的输出文件夹
            if os.path.exists(temp_output_video_path):
                shutil.move(temp_output_video_path, final_output_video_path)
            break  # 匹配到之后就跳出循环，继续下一个视频文件

print("处理完成！")






---





---

